In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import path

import src.lib.utility_classfier_tuning as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer

In [2]:
result_path='/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/deep learning/resample by size/'
filename = "LSTM resample_size gender prediction_0.csv"
prediction = pd.read_csv(path.join(result_path, filename))

y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
y_test_score_male = prediction['test_male_score'][prediction['test_male_score'].notna()]   
y_test_score_female = prediction['test_female_score'][prediction['test_female_score'].notna()]

filename = "LSTM resample_size gender result_0.csv"
result = pd.read_csv(path.join(result_path, filename))

In [3]:
y_test_score_male

0       0.230625
1       0.171617
2       0.234792
3       0.232877
4       0.020797
          ...   
7702    0.188267
7703    0.101517
7704    0.031654
7705    0.002719
7706    0.039809
Name: test_male_score, Length: 7707, dtype: float64

In [4]:
result['y_test_male'][result['y_test_male'].notna()]

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
7702    1.0
7703    0.0
7704    0.0
7705    0.0
7706    0.0
Name: y_test_male, Length: 7707, dtype: float64

In [5]:
def get_result (records, auc_vec, ba_vec, eod_vec, di_vec, y_test, y_score, y_test_1, y_score_1, y_test_2, y_score_2):    
    threshold = 0.1
    ba = thres.calculate_balanced_accuracy(y_test, y_score, threshold)
    auroc = roc_auc_score(y_test, y_score)
    precision, recall, tpr, tnr, pd_overall = thres.calculate_precision_metrics(y_test, y_score,threshold)
    
    ba_male = thres.calculate_balanced_accuracy(y_test_1, y_score_1, threshold)
    precision_male, recall_male, tpr_male, tnr_male, pd_male = thres.calculate_precision_metrics(y_test_1, y_score_1,threshold)
    
    ba_female = thres.calculate_balanced_accuracy (y_test_2, y_score_2, threshold)
    precision_female, recall_female, tpr_female, tnr_female, pd_female = thres.calculate_precision_metrics(y_test_2, y_score_2,threshold)

    eod = fair.get_EOD(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)
    di = fair.get_SP(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)
    
    auc_vec.append(auroc)
    ba_vec.append(ba)
    eod_vec.append(eod)
    di_vec.append(di)

    records.append({
        'auroc': auroc,
        'overall ba test': ba,
        'male ba test': ba_male,
        'female ba test': ba_female,
        'overall precision':precision,
        'overall recall':recall,
        'overall tpr':tpr,
        'overall tnr':tnr,
        'overall pd':pd_overall,
        'male precision':precision_male,
        'male recall':recall_male,
        'male tpr':tpr_male,
        'male tnr':tnr_male,
        'male pd':pd_male,
        'female precision':precision_female,
        'female recall':recall_female,
        'female tpr':tpr_female,
        'female tnr':tnr_female,
        'female pd':pd_female,
        'eod': eod,
        'di': di,
        })

In [6]:
records_lstm_race = []
auc_race = []
ba_race = []
eod_race = []
di_race = []
for i in range(5):
    filename = "LSTM resample_size race prediction_" + str(i) + ".csv"
    prediction = pd.read_csv(path.join(result_path, filename))
    
    y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
    y_test_score_white = prediction['test_white_score'][prediction['test_white_score'].notna()]   
    y_test_score_black = prediction['test_black_score'][prediction['test_black_score'].notna()]
    
    filename_2 = "LSTM resample_size race result_" + str(i) + ".csv"
    result = pd.read_csv(path.join(result_path, filename_2))
    y_test = result['y_test'][result['y_test'].notna()] 
    y_test_white = result['y_test_white'][result['y_test_white'].notna()] 
    y_test_black = result['y_test_black'][result['y_test_black'].notna()] 
    
    get_result(records_lstm_race, auc_race, ba_race, eod_race, di_race, y_test, y_test_score, y_test_white, y_test_score_white, y_test_black, y_test_score_black)
    

True positive rate of class 1 is  0.784
True positive rate of class 2 is  0.688
Positive prediction rate of class 1 is  0.465
Positive prediction rate of class 2 is  0.4
True positive rate of class 1 is  0.746
True positive rate of class 2 is  0.649
Positive prediction rate of class 1 is  0.405
Positive prediction rate of class 2 is  0.329
True positive rate of class 1 is  0.751
True positive rate of class 2 is  0.634
Positive prediction rate of class 1 is  0.409
Positive prediction rate of class 2 is  0.303
True positive rate of class 1 is  0.669
True positive rate of class 2 is  0.614
Positive prediction rate of class 1 is  0.369
Positive prediction rate of class 2 is  0.306
True positive rate of class 1 is  0.72
True positive rate of class 2 is  0.692
Positive prediction rate of class 1 is  0.383
Positive prediction rate of class 2 is  0.339


In [7]:
print(auc_race, np.mean(auc_race), np.std(auc_race))
print(ba_race, np.mean(ba_race), np.std(ba_race))
print(eod_race, np.mean(eod_race), np.std(eod_race))
print(di_race, np.mean(di_race), np.std(di_race))

[0.7253370695596897, 0.7493057778293556, 0.7489126152258704, 0.7298778400533095, 0.7537024114256184] 0.7414271428187688 0.011498384738811021
[0.6730407135808861, 0.686078630539661, 0.6884397047791615, 0.6653798249591001, 0.6862542817106927] 0.6798386311139003 0.009048153312324192
[0.09600000000000009, 0.09699999999999998, 0.11699999999999999, 0.05500000000000005, 0.028000000000000025] 0.07860000000000003 0.03235181602321575
[1.1625, 1.2310030395136777, 1.3498349834983498, 1.2058823529411764, 1.1297935103244836] 1.2158027772555375 0.07554187601956906


In [8]:
records_lstm_gender = []
auc_gender = []
ba_gender = []
eod_gender = []
di_gender = []
for i in range(5):
    filename = "LSTM resample_size gender prediction_" + str(i) + ".csv"
    prediction = pd.read_csv(path.join(result_path, filename))
    
    y_test_score = prediction['test_score'][prediction['test_score'].notna()]   
    y_test_score_male = prediction['test_male_score'][prediction['test_male_score'].notna()]   
    y_test_score_female = prediction['test_female_score'][prediction['test_female_score'].notna()]
    
    filename_2 = "LSTM resample_size gender result_" + str(i) + ".csv"
    result = pd.read_csv(path.join(result_path, filename_2))
    y_test = result['y_test'][result['y_test'].notna()] 
    y_test_male = result['y_test_male'][result['y_test_male'].notna()] 
    y_test_female = result['y_test_female'][result['y_test_female'].notna()] 
    
    get_result(records_lstm_gender, auc_gender, ba_gender, eod_gender, di_gender, y_test, y_test_score, y_test_male, y_test_score_male, y_test_female, y_test_score_female)
    

True positive rate of class 1 is  0.821
True positive rate of class 2 is  0.649
Positive prediction rate of class 1 is  0.536
Positive prediction rate of class 2 is  0.316
True positive rate of class 1 is  0.822
True positive rate of class 2 is  0.66
Positive prediction rate of class 1 is  0.567
Positive prediction rate of class 2 is  0.33
True positive rate of class 1 is  0.822
True positive rate of class 2 is  0.666
Positive prediction rate of class 1 is  0.518
Positive prediction rate of class 2 is  0.318
True positive rate of class 1 is  0.871
True positive rate of class 2 is  0.652
Positive prediction rate of class 1 is  0.628
Positive prediction rate of class 2 is  0.357
True positive rate of class 1 is  0.79
True positive rate of class 2 is  0.602
Positive prediction rate of class 1 is  0.48
Positive prediction rate of class 2 is  0.292


In [9]:
print(auc_gender, np.mean(auc_gender), np.std(auc_gender))
print(ba_gender, np.mean(ba_gender), np.std(ba_gender))
print(eod_gender, np.mean(eod_gender), np.std(eod_gender))
print(di_gender, np.mean(di_gender), np.std(di_gender))

[0.7403910501395952, 0.7331121046100055, 0.7415489796450676, 0.720356884945448, 0.7316901717370321] 0.7334198382154297 0.0075948394361804224
[0.6810616582584035, 0.6727517954003726, 0.6900624484499833, 0.6618436093031936, 0.6770188164222439] 0.6765476655668394 0.009315206604812323
[0.17199999999999993, 0.16199999999999992, 0.15599999999999992, 0.21899999999999997, 0.18800000000000006] 0.17939999999999995 0.022570777567465443
[1.6962025316455698, 1.718181818181818, 1.628930817610063, 1.7591036414565828, 1.6438356164383563] 1.6892508850664778 0.04788692727434309
